In [234]:
geneinfo = read.table("/publicdata/gencode_v19_20151104/gene_info.tsv", sep = "\t", header = T)

In [309]:
peak_coords = read.table("/projects/PPC/analysis/scrna_pilot/scapa/peaks.bed", "\t", header = F)

# Genes that change

In [233]:
# Signature genes

out = "/projects/PPC/analysis/ppc_pilot/data/aggr_live_then_integrate/"

iPSC_markers        = read.table(paste(out, "iPSC_markers.txt", sep = ""))
repl_markers        = read.table(paste(out, "repl_markers.txt", sep = ""))
ppc_markers         = read.table(paste(out, "all.ppc_markers.txt", sep = ""))
Endocrine_markers   = read.table(paste(out, "Endocrine_markers.txt", sep = ""))
mesenchymal_markers = read.table(paste(out, "mesenchymal_markers.txt", sep = ""))
endothelial_markers = read.table(paste(out, "endothelial_markers.txt", sep = ""))
# PPC_1_markers       = read.table(paste(out, "integrated_PPC_1_markers.txt", sep = ""))
# PPC_2_markers       = read.table(paste(out, "integrated_PPC_2_markers.txt", sep = ""))
# PPC_3_markers       = read.table(paste(out, "integrated_PPC_3_markers.txt", sep = ""))
# PPC_4_markers       = read.table(paste(out, "integrated_PPC_4_markers.txt", sep = ""))

In [235]:
# Changes over PPC time

out = "/projects/PPC/analysis/ppc_pilot/data/aggr_live_then_integrate/"

lm_fit_coefs = read.table(paste(out, "ppc_fit_coefs_frazer_ppc_allexpressedgenes.txt", sep = ""))
lm_fit_coefs_sig = lm_fit_coefs[lm_fit_coefs$q_value < 0.05, ]

mfuzz = read.table(paste(out, "mfuzz_k10_b4_memberships.txt", sep = ""))

# Get genes that change over time/cell type

In [267]:
g2 = mfuzz[mfuzz$maxCol == 2 & mfuzz$real == TRUE,]
g6 = mfuzz[mfuzz$maxCol == 6 & mfuzz$real == TRUE,]

In [268]:
n = 100

iPSC_sigs        = data.frame(celltype = "iPSC",        gene_name = rownames(iPSC_markers[iPSC_markers$p_val_adj < 0.05, ])[1:n],                     qval = iPSC_markers[iPSC_markers$p_val_adj < 0.05, "p_val_adj"][1:n]              )
PPC_sigs         = data.frame(celltype = "all_PPC",     gene_name = rownames(ppc_markers[ppc_markers$p_val_adj < 0.05, ])[1:n],                       qval = ppc_markers[ppc_markers$p_val_adj < 0.05, "p_val_adj"][1:n]               )
repl_sigs        = data.frame(celltype = "repl",        gene_name = rownames(repl_markers[repl_markers$p_val_adj < 0.05, ])[1:n],                     qval = repl_markers[repl_markers$p_val_adj < 0.05, "p_val_adj"][1:n]              )
endrocrine_sigs  = data.frame(celltype = "endrocrine",  gene_name = rownames(Endocrine_markers[Endocrine_markers$p_val_adj < 0.05, ])[1:n],           qval = Endocrine_markers[Endocrine_markers$p_val_adj < 0.05, "p_val_adj"][1:n]    )
mesenchyme_sigs  = data.frame(celltype = "mesenchyme",  gene_name = rownames(mesenchymal_markers[mesenchymal_markers$p_val_adj < 0.05, ])[1:n],       qval = mesenchymal_markers[mesenchymal_markers$p_val_adj < 0.05, "p_val_adj"][1:n])
endothelial_sigs = data.frame(celltype = "endothelial", gene_name = rownames(endothelial_markers[endothelial_markers$p_val_adj < 0.05, ])[1:n],       qval = endothelial_markers[endothelial_markers$p_val_adj < 0.05, "p_val_adj"][1:n])
delta_ppc_sigs   = data.frame(celltype = "delta_PPC",   gene_name = lm_fit_coefs_sig[order(-abs(lm_fit_coefs_sig$estimate)), "gene_short_name"][1:n], qval = lm_fit_coefs_sig[order(-abs(lm_fit_coefs_sig$estimate)), "q_value"][1:n])

interesting_genes = rbind(iPSC_sigs, PPC_sigs, repl_sigs, endrocrine_sigs, mesenchyme_sigs, endothelial_sigs, delta_ppc_sigs)

In [269]:
interesting_genes.annotated = merge(interesting_genes, geneinfo, by = "gene_name")

In [270]:
interesting_genes.annotated$gene   = unlist(lapply(interesting_genes.annotated$gene_id, function(x){unlist(strsplit(x, "\\."))[[1]]}))

In [271]:
head(interesting_genes.annotated)

gene_name,celltype,qval,gene_id,gene_type,chrom,start,end,strand,gene_status,source,level,gene
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<chr>
A1CF,endrocrine,7.792998e-239,ENSG00000148584.10,protein_coding,chr10,52559168,52645435,-,KNOWN,HAVANA,2,ENSG00000148584
A2M,endothelial,0.000000e+00,ENSG00000175899.10,protein_coding,chr12,9220259,9268825,-,KNOWN,HAVANA,2,ENSG00000175899
AC027612.6,delta_PPC,1.599904e-04,ENSG00000143429.5,pseudogene,chr2,91805185,91847998,-,KNOWN,HAVANA,2,ENSG00000143429
ACAT2,iPSC,0.000000e+00,ENSG00000120437.7,protein_coding,chr6,160181359,160200144,+,KNOWN,HAVANA,2,ENSG00000120437
ACKR3,endothelial,0.000000e+00,ENSG00000144476.5,protein_coding,chr2,237476429,237491001,+,KNOWN,HAVANA,2,ENSG00000144476
ACTN4,delta_PPC,5.723580e-05,ENSG00000130402.7,protein_coding,chr19,39138288,39222223,+,KNOWN,HAVANA,1,ENSG00000130402


# Alternative poly-Adenylation (APA)

In [284]:
GetGenesWithAPA <- function(f){
    
    apa = read.table(f, head = T)
    apa$gene = unlist(lapply(apa$Gene_ID, function(x){unlist(strsplit(x, "\\."))[[1]]}))
    apa_interesting = apa[apa$gene %in% interesting_genes.annotated$gene, ]
    interesting_genes.annotated_APA = interesting_genes.annotated[interesting_genes.annotated$gene %in% apa_interesting$gene, ]
    interesting_genes.annotated_APA$qval <- NULL
    final = merge(interesting_genes.annotated_APA, apa_interesting)
    final = final[order(final$qval), ]
    final$level = NULL
    final$source = NULL
    final$gene_status = NULL
    final$strand = NULL
    final$gene_type = NULL
    final$gene = NULL
    

    return(final)
    
}

## All cell types

interesting genes: SSTR2

In [303]:
(unique(GetGenesWithAPA("/projects/PPC/analysis/scrna_pilot/scapa/results_peak_pvalues.txt")$gene_name))
head(GetGenesWithAPA("/projects/PPC/analysis/scrna_pilot/scapa/results_peak_pvalues.txt"), 15)

[1] "VIM"      "KIAA1244" "CD59"     "SDC4"     "CCND2"    "MYO6"    
 [7] "CBX5"     "FLRT2"    "PRKX"     "LIMA1"    "CDH6"     "GABRB3"  
[13] "TCF4"     "PRTG"     "TIMP3"    "SSTR2"    "ITGA1"    "STC1"    
[19] "ONECUT2"  "ZKSCAN1"  "FLT1"     "C1orf21"

,gene_name,celltype,gene_id,chrom,start,end,Gene_ID,UTR_ID,Peak_ID,iPSC_PUI,PPC_moderate_1_PUI,PPC_moderate_2_PUI,repl_PUI,mesenchymal_PUI,PPC_early_PUI,PPC_late_PUI,Endocrine_PUI,endothelial_PUI,pval,qval
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,VIM,all_PPC,ENSG00000026025.9,chr10,17270257,17279592,ENSG00000026025.14,ENSG00000026025.14_2,ENSG00000026025.14_2_2,-6.41604902,-1.8772061,-1.9866581,-2.0221703,-2.8770430,-1.7462617,-1.7422103,-2.1934823,-1.3655211,9.939573e-128,1.824906e-124
5,VIM,delta_PPC,ENSG00000026025.9,chr10,17270257,17279592,ENSG00000026025.14,ENSG00000026025.14_2,ENSG00000026025.14_2_2,-6.41604902,-1.8772061,-1.9866581,-2.0221703,-2.8770430,-1.7462617,-1.7422103,-2.1934823,-1.3655211,9.939573e-128,1.824906e-124
29,KIAA1244,endrocrine,ENSG00000112379.8,chr6,138483057,138665800,ENSG00000112379.8,ENSG00000112379.8_1,ENSG00000112379.8_1_2,-0.07703726,-1.6577497,-1.4944057,-1.5008627,-1.6552195,-1.7331771,-1.4373448,-2.0568164,-1.3843585,6.237381e-54,1.123976e-50
11,CD59,all_PPC,ENSG00000085063.10,chr11,33719806,33757991,ENSG00000085063.14,ENSG00000085063.14_1,ENSG00000085063.14_1_3,0.35682705,0.5326969,0.4844615,0.5122494,0.6928914,0.4875188,0.3653482,0.5935872,0.3027130,6.706546e-50,1.203825e-46
14,CD59,delta_PPC,ENSG00000085063.10,chr11,33719806,33757991,ENSG00000085063.14,ENSG00000085063.14_1,ENSG00000085063.14_1_3,0.35682705,0.5326969,0.4844615,0.5122494,0.6928914,0.4875188,0.3653482,0.5935872,0.3027130,6.706546e-50,1.203825e-46
12,CD59,all_PPC,ENSG00000085063.10,chr11,33719806,33757991,ENSG00000085063.14,ENSG00000085063.14_1,ENSG00000085063.14_1_1,0.11108173,0.3860456,0.4098722,0.3599781,0.1609604,0.4225705,0.4819412,0.3851881,0.6746818,1.111197e-48,1.992376e-45
15,CD59,delta_PPC,ENSG00000085063.10,chr11,33719806,33757991,ENSG00000085063.14,ENSG00000085063.14_1,ENSG00000085063.14_1_1,0.11108173,0.3860456,0.4098722,0.3599781,0.1609604,0.4225705,0.4819412,0.3851881,0.6746818,1.111197e-48,1.992376e-45
3,VIM,all_PPC,ENSG00000026025.9,chr10,17270257,17279592,ENSG00000026025.14,ENSG00000026025.14_2,ENSG00000026025.14_2_1,3.35130784,0.9726650,1.0431432,1.0540214,1.7052690,0.9500513,0.8860856,1.2659493,0.7490387,3.617302e-45,6.449649e-42
6,VIM,delta_PPC,ENSG00000026025.9,chr10,17270257,17279592,ENSG00000026025.14,ENSG00000026025.14_2,ENSG00000026025.14_2_1,3.35130784,0.9726650,1.0431432,1.0540214,1.7052690,0.9500513,0.8860856,1.2659493,0.7490387,3.617302e-45,6.449649e-42


# All cell types, no iPSC

In [324]:

peak_coords[peak_coords$V4 == "ENSG00000100836.10_1_2", ]

,V1,V2,V3,V4,V5,V6
,<chr>,<int>,<int>,<chr>,<chr>,<chr>
6949,14,23794292,23794438,ENSG00000100836.10_1_2,.,+


In [313]:


(unique(GetGenesWithAPA("/projects/PPC/analysis/scrna_pilot/scapa/results_peak_pvalues_noipsc.txt")$gene_name))
x = GetGenesWithAPA("/projects/PPC/analysis/scrna_pilot/scapa/results_peak_pvalues_noipsc.txt")
x
x[x$gene_name == "TIMP3", ]

[1] "VIM"      "CD59"     "CCND2"    "SDC4"     "MYO6"     "FLRT2"   
 [7] "CBX5"     "CDH6"     "PRTG"     "TIMP3"    "ONECUT2"  "STC1"    
[13] "ITGA1"    "ZKSCAN1"  "FLT1"     "LIMA1"    "GABRB3"   "KIAA1244"

,gene_name,celltype,gene_id,chrom,start,end,Gene_ID,UTR_ID,Peak_ID,PPC_moderate_1_PUI,PPC_moderate_2_PUI,repl_PUI,mesenchymal_PUI,PPC_early_PUI,PPC_late_PUI,Endocrine_PUI,endothelial_PUI,pval,qval
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,VIM,all_PPC,ENSG00000026025.9,chr10,17270257,17279592,ENSG00000026025.14,ENSG00000026025.14_2,ENSG00000026025.14_2_2,-1.87720614,-1.98665815,-2.022170316,-2.8770430,-1.7462617,-1.7422103,-2.19348234,-1.3655211,3.541073e-82,3.264869e-79
4,VIM,delta_PPC,ENSG00000026025.9,chr10,17270257,17279592,ENSG00000026025.14,ENSG00000026025.14_2,ENSG00000026025.14_2_2,-1.87720614,-1.98665815,-2.022170316,-2.8770430,-1.7462617,-1.7422103,-2.19348234,-1.3655211,3.541073e-82,3.264869e-79
10,CD59,all_PPC,ENSG00000085063.10,chr11,33719806,33757991,ENSG00000085063.14,ENSG00000085063.14_1,ENSG00000085063.14_1_3,0.53269690,0.48446150,0.512249413,0.6928914,0.4875188,0.3653482,0.59358720,0.3027130,1.398270e-50,1.286409e-47
13,CD59,delta_PPC,ENSG00000085063.10,chr11,33719806,33757991,ENSG00000085063.14,ENSG00000085063.14_1,ENSG00000085063.14_1_3,0.53269690,0.48446150,0.512249413,0.6928914,0.4875188,0.3653482,0.59358720,0.3027130,1.398270e-50,1.286409e-47
11,CD59,all_PPC,ENSG00000085063.10,chr11,33719806,33757991,ENSG00000085063.14,ENSG00000085063.14_1,ENSG00000085063.14_1_1,0.38604565,0.40987218,0.359978140,0.1609604,0.4225705,0.4819412,0.38518806,0.6746818,5.432982e-41,4.987477e-38
14,CD59,delta_PPC,ENSG00000085063.10,chr11,33719806,33757991,ENSG00000085063.14,ENSG00000085063.14_1,ENSG00000085063.14_1_1,0.38604565,0.40987218,0.359978140,0.1609604,0.4225705,0.4819412,0.38518806,0.6746818,5.432982e-41,4.987477e-38
3,VIM,all_PPC,ENSG00000026025.9,chr10,17270257,17279592,ENSG00000026025.14,ENSG00000026025.14_2,ENSG00000026025.14_2_1,0.97266497,1.04314322,1.054021413,1.7052690,0.9500513,0.8860856,1.26594928,0.7490387,2.171787e-40,1.991529e-37
6,VIM,delta_PPC,ENSG00000026025.9,chr10,17270257,17279592,ENSG00000026025.14,ENSG00000026025.14_2,ENSG00000026025.14_2_1,0.97266497,1.04314322,1.054021413,1.7052690,0.9500513,0.8860856,1.26594928,0.7490387,2.171787e-40,1.991529e-37
38,CCND2,all_PPC,ENSG00000118971.3,chr12,4382937,4414516,ENSG00000118971.7,ENSG00000118971.7_2,ENSG00000118971.7_2_1,0.24934900,0.29542084,0.276805260,0.3773749,0.1782685,0.4531940,0.12502096,0.1183390,2.515966e-32,2.294561e-29


,gene_name,celltype,gene_id,chrom,start,end,Gene_ID,UTR_ID,Peak_ID,PPC_moderate_1_PUI,PPC_moderate_2_PUI,repl_PUI,mesenchymal_PUI,PPC_early_PUI,PPC_late_PUI,Endocrine_PUI,endothelial_PUI,pval,qval
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
20,TIMP3,endothelial,ENSG00000100234.11,chr22,33197686,33259030,ENSG00000100234.11,ENSG00000100234.11_1,ENSG00000100234.11_1_4,-0.05221064,-0.01103403,-0.005755825,0.2242280,0.2281848,0.3020142,0.1319762,0.9618874,3.716346e-07,0.0002727798
21,TIMP3,endothelial,ENSG00000100234.11,chr22,33197686,33259030,ENSG00000100234.11,ENSG00000100234.11_1,ENSG00000100234.11_1_1,-0.22999776,-0.29343376,-0.302148828,-0.5206058,-0.8340995,-0.6856491,-0.2639525,-0.7910197,6.053878e-06,0.0041166370
19,TIMP3,endothelial,ENSG00000100234.11,chr22,33197686,33259030,ENSG00000100234.11,ENSG00000100234.11_1,ENSG00000100234.11_1_2,0.28220840,0.30446779,0.307904654,0.2963778,0.6059147,0.3836349,0.1319762,-0.1708678,7.678753e-04,0.3870091655


# All cell types (PPCs collapsed), no iPSC

In [299]:
head(GetGenesWithAPA("/projects/PPC/analysis/scrna_pilot/scapa/results_peak_pvalues_noipsc_1ppc.txt"))


,gene_name,celltype,gene_id,chrom,start,end,Gene_ID,UTR_ID,Peak_ID,repl_PUI,mesenchymal_PUI,Endocrine_PUI,endothelial_PUI,PPC_PUI,pval,qval
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,VIM,all_PPC,ENSG00000026025.9,chr10,17270257,17279592,ENSG00000026025.14,ENSG00000026025.14_2,ENSG00000026025.14_2_2,-2.0221703,-2.8770430,-2.1934823,-1.3655211,-1.8055445,2.954809e-72,1.415353e-69
4,VIM,delta_PPC,ENSG00000026025.9,chr10,17270257,17279592,ENSG00000026025.14,ENSG00000026025.14_2,ENSG00000026025.14_2_2,-2.0221703,-2.8770430,-2.1934823,-1.3655211,-1.8055445,2.954809e-72,1.415353e-69
3,VIM,all_PPC,ENSG00000026025.9,chr10,17270257,17279592,ENSG00000026025.14,ENSG00000026025.14_2,ENSG00000026025.14_2_1,1.0540214,1.7052690,1.2659493,0.7490387,0.9343023,1.143728e-39,5.467021e-37
6,VIM,delta_PPC,ENSG00000026025.9,chr10,17270257,17279592,ENSG00000026025.14,ENSG00000026025.14_2,ENSG00000026025.14_2_1,1.0540214,1.7052690,1.2659493,0.7490387,0.9343023,1.143728e-39,5.467021e-37
10,CD59,all_PPC,ENSG00000085063.10,chr11,33719806,33757991,ENSG00000085063.14,ENSG00000085063.14_1,ENSG00000085063.14_1_1,0.3599781,0.1609604,0.3851881,0.6746818,0.4414629,2.036785e-26,9.695097e-24
13,CD59,delta_PPC,ENSG00000085063.10,chr11,33719806,33757991,ENSG00000085063.14,ENSG00000085063.14_1,ENSG00000085063.14_1_1,0.3599781,0.1609604,0.3851881,0.6746818,0.4414629,2.036785e-26,9.695097e-24


# Just PPC

In [323]:
x = read.table("/projects/PPC/analysis/scrna_pilot/scapa/results_peak_pvalues_ppc.txt", header = T)
head(x[order(x$qval), ])

,Gene_ID,UTR_ID,Peak_ID,PPC_moderate_1_PUI,PPC_moderate_2_PUI,PPC_early_PUI,PPC_late_PUI,pval,qval
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
61,ENSG00000100836.10,ENSG00000100836.10_1,ENSG00000100836.10_1_2,-1.1510332,-1.2479847,-1.02777051,-1.2937274,5.280662e-42,1.436340e-39
163,ENSG00000138760.8,ENSG00000138760.8_1,ENSG00000138760.8_1_1,0.4012027,0.4468451,0.47625747,0.7916228,8.429857e-41,2.284491e-38
234,ENSG00000189306.10,ENSG00000189306.10_1,ENSG00000189306.10_1_3,-0.8887040,-1.1920397,-0.72915294,-1.2626832,1.568484e-28,4.234906e-26
121,ENSG00000118971.7,ENSG00000118971.7_2,ENSG00000118971.7_2_1,0.2493490,0.2954208,0.17826848,0.4531940,2.960781e-28,7.964501e-26
12,ENSG00000010244.17,ENSG00000010244.17_1,ENSG00000010244.17_1_4,-0.1631954,-0.2691687,-0.08443981,-0.4988587,1.359535e-25,3.643553e-23
263,ENSG00000213639.9,ENSG00000213639.9_2,ENSG00000213639.9_2_3,-0.1347652,-0.2429008,-0.05273389,-0.3966431,2.403079e-25,6.416221e-23


In [300]:
head(GetGenesWithAPA("/projects/PPC/analysis/scrna_pilot/scapa/results_peak_pvalues_ppc.txt"))


,gene_name,celltype,gene_id,chrom,start,end,Gene_ID,UTR_ID,Peak_ID,PPC_moderate_1_PUI,PPC_moderate_2_PUI,PPC_early_PUI,PPC_late_PUI,pval,qval
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
15,CCND2,all_PPC,ENSG00000118971.3,chr12,4382937,4414516,ENSG00000118971.7,ENSG00000118971.7_2,ENSG00000118971.7_2_1,0.2493490,0.2954208,0.1782685,0.4531940,2.960781e-28,7.964501e-26
18,CCND2,delta_PPC,ENSG00000118971.3,chr12,4382937,4414516,ENSG00000118971.7,ENSG00000118971.7_2,ENSG00000118971.7_2_1,0.2493490,0.2954208,0.1782685,0.4531940,2.960781e-28,7.964501e-26
13,CDH6,all_PPC,ENSG00000113361.8,chr5,31193856,31329253,ENSG00000113361.12,ENSG00000113361.12_2,ENSG00000113361.12_2_2,0.5462501,0.4764069,0.5442138,0.6457763,6.694291e-15,1.753904e-12
10,CDH6,all_PPC,ENSG00000113361.8,chr5,31193856,31329253,ENSG00000113361.12,ENSG00000113361.12_2,ENSG00000113361.12_2_4,0.8001294,0.8723893,0.7502011,0.7353273,2.616038e-13,6.697056e-11
2,VIM,all_PPC,ENSG00000026025.9,chr10,17270257,17279592,ENSG00000026025.14,ENSG00000026025.14_2,ENSG00000026025.14_2_2,-1.8772061,-1.9866581,-1.7462617,-1.7422103,6.980760e-12,1.759152e-09
5,VIM,delta_PPC,ENSG00000026025.9,chr10,17270257,17279592,ENSG00000026025.14,ENSG00000026025.14_2,ENSG00000026025.14_2_2,-1.8772061,-1.9866581,-1.7462617,-1.7422103,6.980760e-12,1.759152e-09


In [259]:
# toString(unique(interesting_genes.annotated[interesting_genes.annotated$gene %in% apa_interesting$gene, ]$gene_id))